<a href="https://colab.research.google.com/github/DiploDatos/AprendizajeProfundo/blob/master/8_automated_hyperparameter_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Búsqueda de Hiperparámetros

Las redes neuronales tienen decenas de hiperparámetros que afectan su arquitectura y proceso de entrenamiento. Más aún, el desempeño final del modelo está condicionado a encontar un conjunto de valores para dichos hiperparámetros exitosos, para una inicialización aleatoria de los pesos dada. Por ello, la exploración de hiperparámetros se vuelve una de las partes más tediosas y críticas del entrenamiento de redes neuronales. Para obtener resultados que sean correctos, significativos y reproducibles es necesario planificar y sistemizar este proceso de búsqueda.

  >  hyper-parameter optimization should be regarded as a formal outer loop in the learning process

Formalmente, este proceso se puede describir como la minimización de la función de pérdida (o maximizar la performance) como si fuera una función de *caja negra* que toma como parámetros los valores de los hiperparámetros:

$$ f(\theta) = loss_\theta(y, \hat{y}) $$
$$ \theta^* = argmin_\theta f(\theta) $$

donde $\theta$ es el conjunto de hiperparámetros del modelo, $loss$ es la pérdida generada entre las etiquetas verdaderas $y$ y las etiquetas generadas por el modelo $\hat{y}$, y $f$ es la función objetivo de la minimización.


Las estrategias principales para la exploración del espacio de hiperparámetros son:
* Búsqueda manual, donde un humano define los valores de cada hiperparámetro.
* Búsqueda por grilla o *grid search*, donde se define un conjunto de valores posibles que puede tomar cada hiperparámetro, y se realiza un experimento por cada combinación posible.
* Búsqueda aleatoria o *random search*, donde se define un rango de valores posibles para cada hiperparámetro, y se elige al azar un valor del rango para cada experimento.
* Búsqueda automátizada, *automated search* o *model-based search*, que es igual a la búsqueda aleatoria pero la selección del valor de cada hiperparámetro está condicionado por los resultados de experimentos anteriores. Para más información ver el paper [*Algorithms for Hyper-Parameter Optimization*](https://proceedings.neurips.cc/paper/2011/file/86e8f7ab32cfd12577bc2619bc635690-Paper.pdf)

En la siguiente imagen, tomada del paper [*Random Search for Hyper-Parameter Optimization*](https://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf), se muestra el impacto de las primeras dos estrategias para un hiperparámetro con alta influencia en el desempeño del modelo final, y otro que sin influencia. No solo require muchas evaluaciones para lograr cobertura, sino que las combinaciones en dónde sólo se varían hiperparámetros no relevantes no recolectan información nueva. El éxito de la búsqueda por grilla depende de que el nivel de granularidad de la grilla cubra adecuadamente los valores relevantes, que son desconocidos a priori.

![Comparación de las exploraciones entre grid search y random search](https://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1531340388/grid_vs_random_jltknd.png)


Para solucionar todos estos problemas, es que se utiliza la **exploración bayesiana**. Este método modela la loss como un Gaussian process, y tiene en cuenta los resultados de los experimentos anteriores para ir construyendo una distribución de probabilidad de la pérdida dados los hiperparámetros:

$$ P(loss | \theta)$$

Para elegir una nueva combinación de hiperparámetros a probar dados los experimentos previos, el algoritmo utiliza una *surrogate function* para aproximar el comportamiento de la pérdida y una *selection function* basada en la mejora esperada. A grandes rasgos, el algoritmo sigue los siguientes pasos:

  1. Encontrar el mejor conjunto de hiperparámetros que maximize la mejora esperada (EI), estimada a través de la *surrogate function*.
  2. Calcular la performance del modelo con la combinación de hiperparámetros elegida. Esto corresponde a evaluar la función objetivo.
  3. Actualizar la forma de la *surrogate function* utilizando el teorema de Bayes para que se ajuste mejor a la verdadera distribución $ P(loss | \theta)$

Afortunadamente, muchos algoritmos de búsqueda están implementados y funcionan como cajas negras. Veremos un ejemplo utilizando la librería Optuna

In [35]:
# If running in colab, you need to update gensim
# !pip install --upgrade gensim

In [1]:
import csv
import functools
import gzip
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import tempfile
import seaborn


from gensim import corpora
from gensim.models import KeyedVectors
from gensim.parsing import preprocessing
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn import metrics
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, IterableDataset
from tqdm.notebook import tqdm, trange


In [2]:
# Ensure version 4.X
import gensim
gensim.__version__

'4.2.0'

## Parte 1: Preprocesamiento del texto

Primero leeremos el dataset como se explica en la notebook 5_cnns.ipynb.

In [3]:
# If necessary, download data
# %%bash
# mkdir data
# curl -L https://cs.famaf.unc.edu.ar/\~ccardellino/resources/diplodatos/glove.6B.50d.txt.gz -o ./data/glove.6B.50d.txt.gz
# curl -L https://cs.famaf.unc.edu.ar/\~ccardellino/resources/diplodatos/imdb_reviews.csv.gz -o ./data/imdb_reviews.csv.gz

In [4]:
class IMDBReviewsDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform
    
    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, item):
        if torch.is_tensor(item):
            item = item.to_list()
        
        item = {
            "data": self.dataset.loc[item, "review"],
            "target": self.dataset.loc[item, "sentiment"]
        }
        
        if self.transform:
            item = self.transform(item)
        
        return item

class RawDataProcessor:
    def __init__(self, 
                 dataset, 
                 ignore_header=True, 
                 filters=None, 
                 vocab_size=50000):
        if filters:
            self.filters = filters
        else:
            self.filters = [
                lambda s: s.lower(),
                preprocessing.strip_tags,
                preprocessing.strip_punctuation,
                preprocessing.strip_multiple_whitespaces,
                preprocessing.strip_numeric,
                preprocessing.remove_stopwords,
                preprocessing.strip_short,
            ]
        
        # Create dictionary based on all the reviews (with corresponding preprocessing)
        self.dictionary = corpora.Dictionary(
            dataset["review"].map(self._preprocess_string).tolist()
        )
        # Filter the dictionary and compactify it (make the indices continous)
        self.dictionary.filter_extremes(no_below=2, no_above=1, keep_n=vocab_size)
        self.dictionary.compactify()
        # Add a couple of special tokens
        self.dictionary.patch_with_special_tokens({
            "[PAD]": 0,
            "[UNK]": 1
        })
        self.idx_to_target = sorted(dataset["sentiment"].unique())
        self.target_to_idx = {t: i for i, t in enumerate(self.idx_to_target)}

    def _preprocess_string(self, string):
        return preprocessing.preprocess_string(string, filters=self.filters)

    def _sentence_to_indices(self, sentence):
        return self.dictionary.doc2idx(sentence, unknown_word_index=1)
    
    def encode_data(self, data):
        return self._sentence_to_indices(self._preprocess_string(data))
    
    def encode_target(self, target):
        return self.target_to_idx[target]
    
    def __call__(self, item):
        if isinstance(item["data"], str):
            data = self.encode_data(item["data"])
        else:
            data = [self.encode_data(d) for d in item["data"]]
        
        if isinstance(item["target"], str):
            target = self.encode_target(item["target"])
        else:
            target = [self.encode_target(t) for t in item["target"]]
        
        return {
            "data": data,
            "target": target,
            "sentence": item["data"]
        }

### Separando el conjunto de validación o *dev*

En deep learning, es **MUY** importante utilizar un conjunto de validación durante la búsqueda de hiperparámetros, que puede ser tomado de la partición de entrenamiento. Esto es independiente de la estrategia de búsqueda que se utilice.

De esta manera, se previene el overfitting indirecto y se cuenta con una partición de datos nunca antes vista para poder evaluar la generalización real del modelo a datos no vistos.


In [5]:
dataset = pd.read_csv("./data/imdb_reviews.csv.gz")
preprocess = RawDataProcessor(dataset)
train_indices, test_indices = train_test_split(dataset.index, test_size=0.2, random_state=42)
train_indices, dev_indices = train_test_split(train_indices, test_size=0.2, random_state=42)
train_dataset = IMDBReviewsDataset(dataset.loc[train_indices].reset_index(drop=True), transform=preprocess)
dev_dataset = IMDBReviewsDataset(dataset.loc[dev_indices].reset_index(drop=True), transform=preprocess)
# We won't use test_dataset until the end!
test_dataset = IMDBReviewsDataset(dataset.loc[test_indices].reset_index(drop=True), transform=preprocess)

In [6]:
class PadSequences:
    def __init__(self, pad_value=0, max_length=100):
        self.pad_value = pad_value
        self.max_length = max_length

    def __call__(self, items):
        data, target = list(zip(*[(item["data"], item["target"]) for item in items]))
        seq_lengths = [len(d) for d in data]

        max_length = self.max_length
        seq_lengths = [min(self.max_length, l) for l in seq_lengths]

        data = [d[:l] + [self.pad_value] * (max_length - l)
                for d, l in zip(data, seq_lengths)]
            
        return {
            "data": torch.LongTensor(data),
            "target": torch.FloatTensor(target)
        }

## Parte 2: Esqueleto de la red neuronal

Definimos el modelo a entrenar.

In [7]:
import torch
import torch.nn as nn

In [8]:
class ImdbLSTM(nn.Module):
    def __init__(self,
                 pretrained_embeddings_path, dictionary, embedding_size,
                 hidden_layer=32,
                 num_layers=1, dropout=0., bias=True,
                 bidirectional=False,
                 freeze_embedings=True):
        
        super(ImdbLSTM, self).__init__()
        output_size = 1
        # Create the Embeddings layer and add pre-trained weights
        embeddings_matrix = torch.randn(len(dictionary), embedding_size)
        embeddings_matrix[0] = torch.zeros(embedding_size)
        with gzip.open(pretrained_embeddings_path, "rt") as fh:
            for line in fh:
                word, vector = line.strip().split(None, 1)
                if word in dictionary.token2id:
                    embeddings_matrix[dictionary.token2id[word]] =\
                        torch.FloatTensor([float(n) for n in vector.split()])
        self.embedding_config = {'freeze': freeze_embedings,
                                  'padding_idx': 0}
        self.embeddings = nn.Embedding.from_pretrained(
            embeddings_matrix, **self.embedding_config)
        
        # Set our LSTM parameters
        self.lstm_config = {'input_size': embedding_size,
                            'hidden_size': hidden_layer,
                            'num_layers': num_layers,
                            'bias': bias,
                            'batch_first': True,
                            'dropout': dropout if num_layers > 1 else 0.0,
                            'bidirectional': bidirectional}
        
        # Set our fully connected layer parameters
        self.linear_config = {'in_features': hidden_layer,
                              'out_features': output_size,
                              'bias': bias}
        
        # Instanciate the layers
        self.lstm = nn.LSTM(**self.lstm_config)
        self.droupout_layer = nn.Dropout(dropout)
        self.classification_layer = nn.Linear(**self.linear_config)
        self.activation = nn.Sigmoid()

    def forward(self, inputs):
        emb = self.embeddings(inputs)
        lstm_out, _ = self.lstm(emb)
        # Take last state of lstm, which is a representation of
        # the entire text
        lstm_out = lstm_out[:, -1, :].squeeze()
        lstm_out = self.droupout_layer(lstm_out)
        predictions = self.activation(self.classification_layer(lstm_out))
        return predictions

Encapsularemos el algoritmo de entrenamiento dentro de una función parametrizable. La función debería devolver los resultados obtenidos.

In [9]:
# Some default values
EPOCHS = 2
MAX_SEQUENCE_LEN = 100

In [10]:
import torch.optim as optim

def train_imbd_model(train_dataset, dev_dataset,
                     pretrained_embeddings_path, dictionary, embedding_size,
                     batch_size=128, max_sequence_len=MAX_SEQUENCE_LEN,
                     hidden_layer=32, dropout=0.,
                     epochs=EPOCHS, lr=0.001, optimizer_class=optim.Adam,
                     verbose=False):

    if verbose:
        print_fn = print
    else:
        print_fn = lambda *x: None
    # We define again the data loaders since this code could run in
    # parallel
    pad_sequeces = PadSequences(max_length=max_sequence_len)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                              collate_fn=pad_sequeces, drop_last=False)
    dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False,
                            collate_fn=pad_sequeces, drop_last=False)

    # We are not going to explore all hyperparameters, only this ones.
    model = ImdbLSTM(pretrained_embeddings_path, dictionary, embedding_size,
                     hidden_layer=hidden_layer, dropout=dropout)

    loss_function = nn.BCELoss()
    optimizer = optimizer_class(model.parameters(), lr)

    history = {
        'train_loss': [],
        'test_loss': [],
        'test_avp': []
    }
    for epoch in range(epochs):
        model.train()
        running_loss = []
        print_fn("Epoch", epoch)
        for idx, batch in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(batch["data"])
            loss_value = loss_function(output.squeeze(), batch["target"])
            loss_value.backward()
            optimizer.step()
            running_loss.append(loss_value.item())
        train_loss = sum(running_loss) / len(running_loss)
        print_fn("\t Final train_loss", train_loss)
        history['train_loss'].append(train_loss)
        
        model.eval()
        running_loss = []
        targets = []
        predictions = []
        for batch in dev_loader:
            output = model(batch["data"])
            running_loss.append(
                loss_function(output.squeeze(), batch["target"]).item()
            )
            targets.extend(batch["target"].numpy())
            # Round up model output to get the predictions.
            # What would happen if you change the activation to tanh?
            predictions.extend(output.squeeze().round().detach().numpy())
        test_loss = sum(running_loss) / len(running_loss)
        avp = metrics.average_precision_score(targets, predictions)
        print_fn("\t Final test_loss", test_loss)
        print_fn("\t Final test_avp", avp)
        history['test_loss'].append(test_loss)
        history['test_avp'].append(avp)
    return history

In [11]:
history = train_imbd_model(
    train_dataset, dev_dataset,
    pretrained_embeddings_path="./data/glove.6B.50d.txt.gz",
    dictionary=preprocess.dictionary, embedding_size=50, verbose=True)

Epoch 0
	 Final train_loss 0.6763483471870423
	 Final test_loss 0.6396069403678651
	 Final test_avp 0.6313642773343966
Epoch 1
	 Final train_loss 0.6783126583099365
	 Final test_loss 0.6633364823129442
	 Final test_avp 0.5721578542649114


In [22]:
history['test_avp'][-1]

0.5721578542649114

## Utilizando Optuna



In [12]:
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances

Optuna pide que tengamos ciertas cosas en nuestro código:
- Una funcion de objetivo a minimizar o maximizar
- Una declaración de hiperparámetros a probar
- Cómo operar dentro de un _trial_
- Estudio

Primero redefinamos el training loop para usar optuna. De esta forma accedemos a una funciones muy útiles de Optuna como el [pruning](https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/003_efficient_optimization_algorithms.html?highlight=pruning#pruning-algorithms).

In [34]:
def optuna_train_imbd_model(
                     trial: optuna.trial, # <--- esto es lo importante
                     train_dataset, dev_dataset,
                     pretrained_embeddings_path, dictionary, embedding_size,
                     batch_size=128, max_sequence_len=MAX_SEQUENCE_LEN,
                     hidden_layer=32, dropout=0.,
                     epochs=EPOCHS, lr=0.001, optimizer_class=optim.Adam,
                     verbose=False):
    if verbose:
        print_fn = print
    else:
        print_fn = lambda *x: None
    # We define again the data loaders since this code could run in
    # parallel
    pad_sequeces = PadSequences(max_length=max_sequence_len)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                              collate_fn=pad_sequeces, drop_last=False)
    dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False,
                            collate_fn=pad_sequeces, drop_last=False)

    # We are not going to explore all hyperparameters, only this ones.
    model = ImdbLSTM(pretrained_embeddings_path, dictionary, embedding_size,
                     hidden_layer=hidden_layer, dropout=dropout)

    loss_function = nn.BCELoss()
    optimizer = optimizer_class(model.parameters(), lr)

    history = {
        'train_loss': [],
        'test_loss': [],
        'test_avp': []
    }
    for epoch in range(epochs):
        model.train()
        running_loss = []
        print_fn("Epoch", epoch)
        for idx, batch in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(batch["data"])
            loss_value = loss_function(output.squeeze(), batch["target"])
            loss_value.backward()
            optimizer.step()
            running_loss.append(loss_value.item())
        train_loss = sum(running_loss) / len(running_loss)
        print_fn("\t Final train_loss", train_loss)
        history['train_loss'].append(train_loss)
        
        model.eval()
        running_loss = []
        targets = []
        predictions = []
        for batch in dev_loader:
            output = model(batch["data"])
            running_loss.append(
                loss_function(output.squeeze(), batch["target"]).item()
            )
            targets.extend(batch["target"].numpy())
            # Round up model output to get the predictions.
            # What would happen if you change the activation to tanh?
            predictions.extend(output.squeeze().round().detach().numpy())
        test_loss = sum(running_loss) / len(running_loss)
        avp = metrics.average_precision_score(targets, predictions)
        print_fn("\t Final test_loss", test_loss)
        print_fn("\t Final test_avp", avp)
        history['test_loss'].append(test_loss)
        history['test_avp'].append(avp)
        
        # Report optimizing value to optuna
        trial.report(history['test_loss'][-1], epoch)
        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    return history

Todo el training es igual excepto por la inclusion del ```trial: optuna.trial```, que es un objeto de Optuna que se encarga de llevar seguimiento de una experimento específico con parámetros muestreados de una distribución.

Al acceder a un trial, optuna nos permite terminar prematuramente experimentos no prometedores con el uso de un Pruner: si el resultado muestra mejorías y es peor a resultados anteriores, se termina el entrenamiento y se mueve al siguiente trial.

Para definir los hiperparámetros para los cuales queremos muestrear valores, debemos usar los métodos ```suggest_*``` del mismo trial. Para este ejemplo usaremos los métodos ```suggest_float``` y ```suggest_categorical``` para muestrear valores para el learning rate, el dropout y el optimizador.

Para más información de los valores que podemos muestrear, consultar la [documentación de Optuna](https://optuna.readthedocs.io/en/v1.1.0/reference/trial.html?highlight=trial).

In [30]:
def objective(trial: optuna.Trial) -> float:
    # Set optuna space search
    optims = {'Adam': optim.Adam,
              'RMSprop': optim.RMSprop}
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam',
                                                             'RMSprop'])
    optimizer_class = optims[optimizer_name]
    dropout = trial.suggest_float("dropout", 0.0, 0.5)
    
    history = optuna_train_imbd_model(
        trial,
        train_dataset, dev_dataset,
        pretrained_embeddings_path="./data/glove.6B.50d.txt.gz",
        dictionary=preprocess.dictionary, embedding_size=50,
        lr=lr,
        optimizer_class=optimizer_class,
        dropout=dropout)
    
    # This is the value that will be minimized!            
    return history['test_loss'][-1]

In [35]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5, timeout=600)

[I 2022-09-23 12:34:24,717] A new study created in memory with name: no-name-4476815c-d0cd-4152-806a-d13d161c8761
[I 2022-09-23 12:36:55,526] Trial 0 finished with value: 0.6930831869443258 and parameters: {'lr': 0.006084575186402038, 'optimizer': 'Adam', 'dropout': 0.27568295002047466}. Best is trial 0 with value: 0.6930831869443258.
[I 2022-09-23 12:39:02,772] Trial 1 finished with value: 0.6128588924332271 and parameters: {'lr': 0.0007412787605150763, 'optimizer': 'Adam', 'dropout': 0.391211136945459}. Best is trial 1 with value: 0.6128588924332271.
[I 2022-09-23 12:40:48,294] Trial 2 finished with value: 0.6266448488311162 and parameters: {'lr': 0.061106317059916655, 'optimizer': 'RMSprop', 'dropout': 0.2329279707931622}. Best is trial 1 with value: 0.6128588924332271.
[I 2022-09-23 12:42:21,124] Trial 3 finished with value: 0.692151961818574 and parameters: {'lr': 2.7776045098870157e-05, 'optimizer': 'Adam', 'dropout': 0.20023184547822054}. Best is trial 1 with value: 0.6128588924

In [36]:
from optuna.trial import TrialState

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
    
fig = optuna.visualization.plot_param_importances(study)
fig.show()

Study statistics: 
  Number of finished trials:  5
  Number of pruned trials:  0
  Number of complete trials:  5
Best trial:
  Value:  0.4476667179001702
  Params: 
    lr: 0.03382727347205365
    optimizer: Adam
    dropout: 0.04125212145580892


## Recomendaciones finales

* No es necesario realizar la búsqueda de hiperparámetros sobre el conjunto de datos entero, ni entrenar el clasificador durante todas las epocas hasta que comienza a diverger. Se puede utilizar para encontrar los espacios más prometedores de valores posibles, y luego realizar una segunda búsqueda con con menos iteraciones pero con el proceso de entrenamiento completo.
* No realizar la búsqueda utilizando notebooks, sino scripts.
* Combinar Optuna con mlflow para un registro de los resultados ordenado.
* Modificar el training loop para guardar los modelos para no tener que reentrenar el modelo de mejores parámetros.